# Install dependencies

In [1]:
!pip install transformers datasets torch scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from datasets import Dataset
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import random
import os
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# Load Data

In [4]:
df=pd.read_csv('/kaggle/input/ai-text/ai_press_releases.csv')
df=df.dropna()
human=df['non_chat_gpt_press_release'].to_list()
ai=df['chat_gpt_generated_release'].to_list()
labels=[0 if i<len(ai) else 1 for i in range(len(ai)+len(human))]
ai.extend(human)
texts=ai
# 1) 먼저 train_temp(80%)와 test(20%) 분할
texts_train_val, texts_test, labels_train_val, labels_test = train_test_split(
    texts,
    labels,
    test_size=0.2,       # 전체의 20%
    random_state=42,
    stratify=labels      # 레이블 비율 유지
)

# 2) train_temp을 다시 train(75% of temp → 60% 전체)와 val(25% of temp → 20% 전체)로 분할
texts_train, texts_val, labels_train, labels_val = train_test_split(
    texts_train_val,
    labels_train_val,
    test_size=0.25,      # train_temp의 25% → 전체의 0.2
    random_state=42,
    stratify=labels_train_val
)

print(f"Train: {len(texts_train)} samples")
print(f"Valid: {len(texts_val)} samples")
print(f"Test : {len(texts_test)} samples")

Train: 13898 samples
Valid: 4633 samples
Test : 4633 samples


# Load Model

In [5]:
# 2. Sentence split
def split_sentences(paragraph: str):
    return [s.strip() for s in paragraph.split('. ') if s.strip()]

# 3. Dataset
class ParagraphDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_sents=16, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_sents = max_sents
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        para = self.texts[i]
        label = torch.tensor(self.labels[i], dtype=torch.float)
        sents = split_sentences(para)[:self.max_sents]
        encs = [self.tokenizer(s, truncation=True, padding='max_length',
                               max_length=self.max_len, return_tensors='pt')
                for s in sents]
        # pad sentences
        pad_n = self.max_sents - len(encs)
        input_ids = torch.stack([e['input_ids'].squeeze(0) for e in encs] +
                                [torch.zeros(self.max_len, dtype=torch.long)]*pad_n)
        attn_mask = torch.stack([e['attention_mask'].squeeze(0) for e in encs] +
                                [torch.zeros(self.max_len, dtype=torch.long)]*pad_n)
        return input_ids, attn_mask, label

# 4. Model: frozen encoder + attention + classifier
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class HierAttnClassifier(nn.Module):
    def __init__(self,
                 base_model_name="/kaggle/input/robertector/transformers/sentences/1/checkpoint-epoch3",
                 max_sents=16,
                 hidden=768,
                 heads=4):
        super().__init__()
        # 1) Load your fine‑tuned SequenceClassification model
        self.full_model = AutoModelForSequenceClassification.from_pretrained(
            base_model_name, output_hidden_states=True, return_dict=True
        )
        # 2) Freeze all its parameters
        for p in self.full_model.parameters():
            p.requires_grad = False

        # 3) Multi‑Head Attention on the CLS embeddings
        self.attn = nn.MultiheadAttention(embed_dim=hidden,
                                          num_heads=heads,
                                          batch_first=True)
        # 4) Final MLP head after attention
        self.classifier = nn.Sequential(
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, input_ids, attention_mask):
        b, s, l = input_ids.size()
        # flatten to (b*s, l)
        flat_ids   = input_ids.view(b * s, l)
        flat_mask  = attention_mask.view(b * s, l)
        # 5) Run through RoBERTector; we asked for hidden_states
        outputs = self.full_model(
            input_ids=flat_ids,
            attention_mask=flat_mask,
        )
        # 6) Grab the last hidden layer states: outputs.hidden_states is a tuple
        #    where hidden_states[-1] is (batch, seq_len, hidden)
        last_hid = outputs.hidden_states[-1]        
        # CLS is token 0
        cls_embs = last_hid[:, 0, :].view(b, s, -1)  # (b, s, hidden)

        # 7) Self‑attention over the s sentence embeddings
        attn_out, _ = self.attn(cls_embs, cls_embs, cls_embs)  # (b, s, hidden)

        # 8) Pool and classify
        doc_emb = attn_out.mean(dim=1)                       # (b, hidden)
        logits = self.classifier(doc_emb).squeeze(-1)        # (b,)
        return logits


In [10]:
# 5. Prepare data, loaders, model, optimizer
model_path = "/kaggle/input/robertector/transformers/sentences/1/checkpoint-epoch3"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer from the directory
# This reads files like tokenizer.json and tokenizer_config.json
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model from the directory
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
dataset = ParagraphDataset(texts, labels, tokenizer)
n = len(dataset)

train_n = int(0.6*n); val_n = int(0.2*n); test_n = n - train_n - val_n
train_ds, val_ds, test_ds = random_split(dataset, [train_n, val_n, test_n])
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=64, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=64, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierAttnClassifier().to(device)
opt = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()


In [ ]:
from tqdm.auto import tqdm

num_epochs = 6
os.makedirs('/kaggle/working/ckpts', exist_ok=True)

for epoch in range(1, num_epochs + 1):
    # ── TRAIN ───────────────────────────────────────────────
    model.train()
    train_loss_sum = 0.0
    train_steps    = 0
    loop = tqdm(train_loader, desc=f"Train E{epoch}")
    for ids, mask, lbl in loop:
        ids, mask, lbl = ids.to(device), mask.to(device), lbl.to(device)
        opt.zero_grad()
        logits = model(ids, mask)
        loss   = criterion(logits, lbl)
        loss.backward()
        opt.step()

        train_loss_sum += loss.item()
        train_steps    += 1
        # tqdm 바에 현재 배치 손실 표시
        loop.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = train_loss_sum / train_steps
    print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f}")

    # ── VALIDATION ─────────────────────────────────────────
    model.eval()
    val_loss_sum = 0.0
    preds, trues = [], []
    with torch.no_grad():
        for ids, mask, lbl in val_loader:
            ids, mask, lbl = ids.to(device), mask.to(device), lbl.to(device)
            logits = model(ids, mask)
            loss   = criterion(logits, lbl)
            val_loss_sum += loss.item()
            preds += (torch.sigmoid(logits) > 0.5).cpu().int().tolist()
            trues += lbl.cpu().int().tolist()
    avg_val_loss = val_loss_sum / len(val_loader)
    acc = accuracy_score(trues, preds)
    f1  = f1_score(trues, preds)
    print(f"Epoch {epoch} | Val Loss: {avg_val_loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f}")

    # ── CHECKPOINT SAVE ────────────────────────────────────
    checkpoint_path = f"/kaggle/working/ckpts/epoch{epoch}.pt"
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Saved checkpoint: {checkpoint_path}")

# ── FINAL TEST ────────────────────────────────────────────
model.load_state_dict(torch.load('/kaggle/working/ckpts/epoch6.pt'))
model.eval()
preds, trues = [], []
with torch.no_grad():
    for ids, mask, lbl in test_loader:
        ids, mask, lbl = ids.to(device), mask.to(device), lbl.to(device)
        logits = model(ids, mask)
        preds += (torch.sigmoid(logits) > 0.5).cpu().int().tolist()
        trues += lbl.cpu().int().tolist()
acc = accuracy_score(trues, preds)
f1  = f1_score(trues, preds)
print(f"Test Acc {acc:.4f} | F1 {f1:.4f}")

Train E1:   0%|          | 0/218 [00:00<?, ?it/s]

Epoch 1 | Train Loss: 0.0283
Epoch 1 | Val Loss: 0.0388 | Acc: 0.9877 | F1: 0.9877
Saved checkpoint: /kaggle/working/ckpts/epoch1.pt


Train E2:   0%|          | 0/218 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e71bb927a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e71bb927a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 16

In [13]:
# 3) Obtain attention weights for a sample paragraph
paragraph = "Enter your paragraph here. It can be several sentences. The model will attend to each."
cls_embs, sentences = get_sentence_embeddings(paragraph)
with torch.no_grad():
    doc_emb, attn_weights = model(cls_embs)

# 4) Plot attention heatmap
weights = attn_weights[0].cpu().numpy()  # shape (n_sent, n_sent)
plt.figure(figsize=(6,6))
plt.imshow(weights, aspect='auto')
plt.xlabel('Key Sentence Index')
plt.ylabel('Query Sentence Index')
plt.title('Hierarchical Attention Weights')
plt.colorbar()
plt.xticks(range(len(sentences)), range(len(sentences)))
plt.yticks(range(len(sentences)), range(len(sentences)))
plt.show()

# 5) Sentence importance scores (mean over queries)
importance = weights.mean(axis=0)
top_idx = importance.argsort()[-3:][::-1]
print("Top 3 important sentences:")
for idx in top_idx:
    print(f"{idx}: {sentences[idx]}")

# 6) t-SNE visualization of document embeddings (multiple paragraphs example)
# Suppose you have a list of paragraphs: docs = [p1, p2, ...]
# Here we illustrate with a small list
docs = [paragraph, "Another example paragraph. It has different style."]  # replace with your test set
embs = []
labels = [0,1]  # example labels: 0=human,1=AI
for p in docs:
    cls_embs, _ = get_sentence_embeddings(p)
    with torch.no_grad():
        doc_emb, _ = model(cls_embs)
    embs.append(doc_emb.cpu().numpy().squeeze())
embs = torch.tensor(embs).numpy()
tsne = TSNE(n_components=2, random_state=42)
proj = tsne.fit_transform(embs)
plt.figure()
plt.scatter(proj[:,0], proj[:,1])
for i, label in enumerate(labels):
    plt.annotate(str(label), (proj[i,0], proj[i,1]))
plt.title('t-SNE of Document Embeddings')
plt.show()


NameError: name 'get_sentence_embeddings' is not defined